<a href="https://colab.research.google.com/github/ElenaSerbuValentina/Apriori_spark/blob/main/Apriori_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PYSPARK SETUP


In [ ]:
# Download Java Virtual Machine (JVM)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
# Unzip the file
!tar xf spark-3.4.0-bin-hadoop3.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.4.0-bin-hadoop3'

In [ ]:
# Install library for finding Spark
!pip install -q findspark
# Import the libary
import findspark
# Initiate findspark
findspark.init()
# Check the location for Spark
findspark.find()

'/content/spark-3.4.0-bin-hadoop3'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
conf = SparkConf().setAppName('test_mba')
spark = SparkSession.builder.enableHiveSupport().config(conf=conf).getOrCreate()
#spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

# DATA IMPORT

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import os

In [ ]:
#insert your kaggle keys
os.environ['KAGGLE_USERNAME'] = "eluska"
os.environ['KAGGLE_KEY'] = "91ee9b34765d3cbbbe802cf7b1bd16cb"

In [ ]:
!kaggle datasets download -d xhlulu/medal-emnlp --unzip

100% 6.81G/6.82G [01:32<00:00, 62.5MB/s]
100% 6.82G/6.82G [01:32<00:00, 78.8MB/s]


# IMPORTS

In [ ]:
!git clone https://github.com/ElenaSerbuValentina/Apriori_spark.git

In [37]:
import itertools
from itertools import combinations, product
from pyspark.sql import DataFrame
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

In [38]:
from Apriori_spark.utils.functions import Preprocess, apriori

# DATA PREPARATION

In [39]:
#read data
df = spark.read.csv('/content/full_data.csv', header=True, inferSchema=True)

In [40]:
#data sampling
df = df.sample(fraction=0.0001,seed= 143)

In [41]:
#takes conversation column from rdd
rdd = df.rdd.map(lambda x:x[0])

In [42]:
rdd.take(1)

['anaerobically digested dewatered sludge to cm thick was applied to a forest clearcut as a fertilizer source in northwest washington on gravelly glacial outwash soil this sludge is not microbiologically sterile and may contain pathogenic organisms fecal coliform bacterial counts in sludge applied in summer july fell from x to g in days and to g in days dieoff appeared more rapid in winter januaryapplied sludge when colnts fell from x to g in days initial death rates were related to sludge temperature moisture ph PCS composition and microbial competition aftergrowth of fecal coliforms occurred in warm summer and fall months but counts were of similar magnitude to background levels in forest soils where a maximum count of g was recorded total coliform counts in fresh sludge ranged from x to x g numbers stabilized at to g in spring fall and summer with lower numbers in winter both total and fecal bacteria moved from the sludge to the soil beneath but few penetrated past the first cm the 

In [43]:
#define total numer of transactions
total_count = rdd.count()
total_count

1438

In [44]:
#preprocessing rdd (INITIAL STAGE)
rdd_preprocessed = rdd.map(Preprocess.preprocess)

In [45]:
#encode rdd and get dictionary for decoding the encoded results
#initialize class for running Apriori preprocessing
processor = Preprocess()
#hash the items to integers in order to improve algorithm's running time
rdd_encoded, num2word = processor.encoder(rdd_preprocessed)

#initialize apriori class with minimum support, maximum size of baskets , minimum confidence to generate association rules and the vocabulary
#that maps numbers to words in order to decode the final frequent itemsets
APRIORI = apriori(minSupportpercent =0.1,total_transactions=total_count, maximumBasketSize=3, min_confidence = 0.5, vocab= num2word)

In [46]:
#in order to improve performance
rdd = rdd_encoded
rdd.cache()

PythonRDD[105] at RDD at PythonRDD.scala:53

# FIND SINGLETONS

In [47]:
minSupport = total_count*0.1
singletons = rdd.flatMap(list)\
                 .map(lambda word: (word,1))\
                 .reduceByKey(lambda y,x: x+y)\
                 .filter(lambda t: t[1]>= minSupport)

In [49]:
words_singleton = singletons.take(2)
words_singleton = [(num2word[t[0]],t[1]) for t in words_singleton]

In [50]:
words_singleton

[('significantly', 165), ('analysis', 206)]

# FIND FREQUENT PAIRS

In [51]:
singletons = singletons.map(lambda x: (x[0]))
candidates1 = list(combinations(singletons.toLocalIterator(),2))
minSupport_2 = minSupport/4
#filtering phase to select real frequent pairs
combined_2 = rdd.flatMap(lambda sentence: [(tuple(candidate),1) for candidate in candidates1 if set(list(candidate)).issubset(set(sentence))])\
                .reduceByKey(lambda y,x:x+y)\
                .filter(lambda item : item[1]>= minSupport_2)

In [52]:
words_2 = combined_2.take(2)
words_2 = [(num2word[t[0]],t[1]) if type(t[0])==int else (tuple([num2word[element] for element in t[0]]),t[1]) for t in words_2]

In [53]:
words_2

[(('increased', 'two'), 38), (('high', 'found'), 39)]

# FIND FREQUENT TRIPLETS

In [54]:
#compute candidates for frequent triplets
k=3
candidates = APRIORI.getCombinations(combined_2,k)

In [55]:
minSupport_3 = minSupport/5
combined_3 = rdd.flatMap(lambda sentence: [(tuple(candidate),1) for candidate in candidates if set(list(candidate)).issubset(set(sentence))])\
                .reduceByKey(lambda y,x:x+y)\
                .filter(lambda item : item[1]> minSupport_3)

In [56]:
word_3 = combined_3.take(2)
word_3 = [(num2word[t[0]],t[1]) if type(t[0])==int else (tuple([num2word[element] for element in t[0]]),t[1]) for t in word_3]

In [57]:
word_3

[(('cell', 'also', 'study'), 33), (('cell', 'effect', 'result'), 37)]

# GENERALIZED APRIORI ALGORITHM

In [58]:
#run the generalized Apriori algorithm on my processed rdd
freq_rdd =APRIORI.Apriori(support = sc.parallelize([]), rdd=rdd)

---set up complete---
---singletones found!---
---first candidates found!---
starting 2 items in basket loop
added frequent baskets with 2 items
computing candidates for next iteration
found candidates for 3 items in basket
starting 3 items in basket loop
added frequent baskets with 3 items


In [59]:
results = freq_rdd.collect()

In [60]:
#decode results which are hashed to integers
decoded = [(num2word[t[0]],t[1]) if type(t[0])==int else (tuple([num2word[element] for element in t[0]]),t[1]) for t in results]

In [61]:
decoded

[('significantly', 165),
 ('analysis', 206),
 ('level', 208),
 ('disease', 174),
 ('patient', 335),
 ('compared', 177),
 ('factor', 144),
 ('cell', 315),
 ('time', 178),
 ('protein', 224),
 ('showed', 170),
 ('increased', 185),
 ('case', 157),
 ('high', 191),
 ('result', 375),
 ('associated', 169),
 ('t0', 177),
 ('however', 223),
 ('used', 229),
 ('found', 193),
 ('one', 212),
 ('may', 267),
 ('p', 167),
 ('well', 155),
 ('system', 164),
 ('using', 260),
 ('present', 183),
 ('different', 207),
 ('also', 295),
 ('human', 151),
 ('activity', 215),
 ('significant', 170),
 ('treatment', 251),
 ('method', 149),
 ('role', 176),
 ('mechanism', 150),
 ('change', 155),
 ('effect', 313),
 ('gene', 149),
 ('c', 157),
 ('clinical', 152),
 ('show', 144),
 ('expression', 149),
 ('study', 419),
 ('observed', 179),
 ('two', 245),
 ('response', 154),
 ('control', 167),
 ('data', 168),
 (('increased', 'two'), 38),
 (('high', 'found'), 39),
 (('patient', 'clinical'), 78),
 (('showed', 'gene'), 33),
 (('

# ASSOCIATION RULES

In [62]:
#generate association rules
rules_df = APRIORI.generate_association_rules(results, to_decode=True)

In [63]:
rules_df.head(20)

,antecedent,consequent,support,confidence,interest
0,"(case, treatment)","(patient,)",0.014604,0.700000,0.467038
1,"(analysis, expression)","(cell,)",0.013908,0.666667,0.447612
2,"(also, expression)","(cell,)",0.023644,0.641509,0.422455
3,"(disease, treatment)","(patient,)",0.016690,0.631579,0.398616
5,"(expression,)","(cell,)",0.062587,0.604027,0.384973
7,"(analysis, expression)","(protein,)",0.012517,0.600000,0.444228
8,"(case, clinical)","(patient,)",0.013213,0.593750,0.360788
11,"(protein, system)","(cell,)",0.011822,0.586207,0.367153
13,"(one, present)","(study,)",0.015994,0.575000,0.283623
15,"(effect, expression)","(cell,)",0.021558,0.574074,0.355020


In [ ]:
# save rules to csv
rules_df.to_csv('/content/rules.csv')